## Deribit API Request  - For BTC and ETH Options Data
#### ALL Options data 

### Specfic Settlement Period ( Day, Week, Month) 

In [ ]:
import pandas as pd
import json
import requests
import re
from datetime import datetime
from tqdm import tqdm
import concurrent.futures
import time 

# Functions
def get_option_name_and_settlement(coin):
    """
    :param coin: crypto-currency coin name ('BTC', 'ETH')
    :return: 2 lists:
                        1. list of traded options for the selected coin;
                        2. list of settlement period for the selected coin.
    """
    r = requests.get("https://test.deribit.com/api/v2/public/get_instruments?currency=" + coin + "&kind=option")
    result = json.loads(r.text)
    # Get option name
    name = pd.json_normalize(result['result'])['instrument_name']
    name = list(name)

    # Get option settlement period
    settlement_period = pd.json_normalize(result['result'])['settlement_period']
    settlement_period = list(settlement_period)

    return name, settlement_period 

def extract_details(instrument_name, coin):
    """
    Extract expiration date, strike price, and option type (call or put) from instrument name.
    Adjusts to include selected coin (e.g., 'BTC' or 'ETH').
    """
    match = re.match(fr"{coin}-(\d+[A-Z]{{3}}\d+)-(\d+)-([CP])", instrument_name)
    if match:
        expiration_date = match.group(1)
        strike_price = match.group(2)
        option_type = 'Call' if match.group(3) == 'C' else 'Put'
        return expiration_date, strike_price, option_type
    return None, None, None

def fetch_option_data(option_name):
    """Fetch the option data for a given option name with a small delay to avoid rate limiting, and select only specific columns."""
    time.sleep(0.1)  # Add a short delay to avoid hitting rate limits
    r = requests.get(f'https://test.deribit.com/api/v2/public/get_order_book?instrument_name={option_name}')
    result = json.loads(r.text)
    
    # Normalize the JSON data and filter for required columns
    df = pd.json_normalize(result['result'])
    selected_columns = ["instrument_name", "mark_price", "underlying_price", "mark_iv", "greeks.vega"]
    return df[selected_columns]

def get_option_data(coin, settlement_per):
    # Get option name and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin)
    # Filter options that have the specified settlement period
    coin_name_filtered = [coin_name[i] for i in range(len(coin_name)) if settlement_period[i] == settlement_per]
    # Initialize progress bar
    pbar = tqdm(total=len(coin_name_filtered))

    # Fetch data concurrently using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_option = {executor.submit(fetch_option_data, name): name for name in coin_name_filtered}
        coin_df = []
        for future in concurrent.futures.as_completed(future_to_option):
            try:
                data = future.result()
                data['settlement_period'] = settlement_per
                coin_df.append(data)
            except Exception as exc:
                print(f'Error fetching data: {exc}')
            pbar.update(1)

    # Finalize DataFrame
    if len(coin_df) > 0:
        coin_df = pd.concat(coin_df)
    # Extract expiration date, strike price, and option type
    coin_df['Expiration Date'], coin_df['Strike Price'], coin_df['Option Type'] = zip(*coin_df['instrument_name'].apply(lambda x: extract_details(x, coin)))
    # Calculate time to expiration
    today = datetime.today()
    coin_df['Time to Expiration'] = coin_df['Expiration Date'].apply(lambda x: (datetime.strptime(x, '%d%b%y') - today).days / 365 if x else None)
    # Select the final columns
    final_columns = ["instrument_name", "Option Type", 'mark_price', 'underlying_price', 'mark_iv', 'greeks.vega', 'Expiration Date', 'Strike Price', 'Time to Expiration']
    coin_df = coin_df[final_columns]
    coin_df.to_csv('data/data.csv', index=False)
    pbar.close()
    return coin_df


Unique settlement periods available: {'day', 'week', 'month'}
Filtered option names: ['BTC-29NOV24-32000-C', 'BTC-29NOV24-32000-P', 'BTC-29NOV24-34000-C', 'BTC-29NOV24-34000-P', 'BTC-29NOV24-36000-C', 'BTC-29NOV24-36000-P', 'BTC-29NOV24-38000-C', 'BTC-29NOV24-38000-P', 'BTC-29NOV24-40000-C', 'BTC-29NOV24-40000-P', 'BTC-29NOV24-42000-C', 'BTC-29NOV24-42000-P', 'BTC-29NOV24-43000-C', 'BTC-29NOV24-43000-P', 'BTC-29NOV24-44000-C', 'BTC-29NOV24-44000-P', 'BTC-29NOV24-45000-C', 'BTC-29NOV24-45000-P', 'BTC-29NOV24-46000-C', 'BTC-29NOV24-46000-P', 'BTC-29NOV24-47000-C', 'BTC-29NOV24-47000-P', 'BTC-29NOV24-48000-C', 'BTC-29NOV24-48000-P', 'BTC-29NOV24-49000-C', 'BTC-29NOV24-49000-P', 'BTC-29NOV24-50000-C', 'BTC-29NOV24-50000-P', 'BTC-29NOV24-51000-C', 'BTC-29NOV24-51000-P', 'BTC-29NOV24-52000-C', 'BTC-29NOV24-52000-P', 'BTC-29NOV24-53000-C', 'BTC-29NOV24-53000-P', 'BTC-29NOV24-54000-C', 'BTC-29NOV24-54000-P', 'BTC-29NOV24-55000-C', 'BTC-29NOV24-55000-P', 'BTC-29NOV24-56000-C', 'BTC-29NOV24-5600

  3%|▎         | 22/634 [00:03<00:44, 13.61it/s]

In [2]:
data = get_option_data('BTC','day')

data['Strike Price'] = pd.to_numeric(data['Strike Price'], errors='coerce').astype('float64')

100%|██████████| 130/130 [00:16<00:00,  7.68it/s]


In [3]:
data

,instrument_name,Option Type,mark_price,underlying_price,mark_iv,greeks.vega,Expiration Date,Strike Price,Time to Expiration
0,BTC-31OCT24-69000-P,Put,0.0000,72368.780,23.85,0.00000,31OCT24,69000.0,0.000000
0,BTC-31OCT24-69000-C,Call,0.0466,72368.780,23.85,0.00054,31OCT24,69000.0,0.000000
0,BTC-31OCT24-66500-C,Call,0.0811,72368.780,87.89,0.05015,31OCT24,66500.0,0.000000
0,BTC-31OCT24-66500-P,Put,0.0001,72368.780,87.89,0.65823,31OCT24,66500.0,0.000000
0,BTC-31OCT24-66000-P,Put,0.0001,72368.780,93.55,0.57827,31OCT24,66000.0,0.000000
...,...,...,...,...,...,...,...,...,...
0,BTC-24JAN25-100000-P,Put,0.3615,74586.128,54.73,89.96469,24JAN25,100000.0,0.232877
0,BTC-24JAN25-100000-C,Call,0.0208,74586.128,54.73,89.96469,24JAN25,100000.0,0.232877
0,BTC-24JAN25-105000-P,Put,0.4224,74586.128,54.73,73.98251,24JAN25,105000.0,0.232877
0,BTC-24JAN25-110000-P,Put,0.4850,74586.128,54.73,59.49364,24JAN25,110000.0,0.232877


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 633 entries, 0 to 0
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   instrument_name     633 non-null    object 
 1   Option Type         633 non-null    object 
 2   mark_price          633 non-null    float64
 3   underlying_price    633 non-null    float64
 4   mark_iv             633 non-null    float64
 5   greeks.vega         633 non-null    float64
 6   Expiration Date     633 non-null    object 
 7   Strike Price        633 non-null    float64
 8   Time to Expiration  633 non-null    float64
dtypes: float64(6), object(3)
memory usage: 49.5+ KB


In [20]:
data.isnull().sum()

instrument_name       0
Option Type           0
mark_price            0
underlying_price      0
mark_iv               0
greeks.vega           0
Expiration Date       0
Strike Price          0
Time to Expiration    0
dtype: int64

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 633 entries, 0 to 0
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   instrument_name     633 non-null    object 
 1   Option Type         633 non-null    object 
 2   mark_price          633 non-null    float64
 3   underlying_price    633 non-null    float64
 4   mark_iv             633 non-null    float64
 5   greeks.vega         633 non-null    float64
 6   Expiration Date     633 non-null    object 
 7   Strike Price        633 non-null    float64
 8   Time to Expiration  633 non-null    float64
dtypes: float64(6), object(3)
memory usage: 49.5+ KB


In [18]:
data.to_csv('data/data.csv', index=False)